In [1]:
using Pkg
Pkg.activate(".")

  Activating new environment at `~/Documents/Github2/Physics-215-Julia/Session 4 - Fast function calls/Project.toml`


In [45]:
using BenchmarkTools
using DataFrames

### KR1: Benchmarked at least two (2) different implementation of the same function or process (e.g. raising each element of an array to some power `p`, random array may be used) that utilizes some parameter that can be considered a constant or declared globally. Typical methods: (1) Global variable, (2) Constant global variable, and (3) Named parameter variable.

Now let us create two functions whose input are both arrays containing random numbers and the result is the sum of the elements that were multiplied to some value `p` and `p2`. The difference of the two functions is that the first function `multiply_array` uses a global variable `p` while the other function `multiply_array2` uses a constant global variable `p2`.

In [2]:
p = 2 

function multiply_array(x::Vector{Float64})
    s =  zero(eltype(x))
    for y in x
        s = s + y*p
    end
    return s
end

multiply_array (generic function with 1 method)

In [3]:
const p2 = 2

function multiply_array2(x::Vector{Float64})
    s = zero(eltype(x))
    for y in x
        s = s + y*p2
    end
    return s
end

multiply_array2 (generic function with 1 method)

In [4]:
random_array = rand(1000);

In [51]:
time_global = @benchmark multiply_array($random_array)

BenchmarkTools.Trial: 7797 samples with 1 evaluation.
 Range (min … max):   55.027 μs … 120.805 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     111.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   543.932 μs ±   3.086 ms  ┊ GC (mean ± σ):  2.78% ± 3.11%

  █▄▂▁                                                          ▁
  ██████▇▇▇▆▇▆▆▅▅▄▅▅▅▄▅▄▄▅▅▄▁▃▃▄▄▄▅▄▅▄▄▃▃▆▃▄▁▅▄▄▄▅▄▄▁▄▄▅▄▄▅▄▄▅▅ █
  55 μs         Histogram: log(frequency) by time       12.3 ms <

 Memory estimate: 46.88 KiB, allocs estimate: 3000.

In [52]:
time_constant = @benchmark multiply_array2($random_array)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.098 μs …  2.856 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.148 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.094 μs ± 63.949 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁▁ ▁                                                      ▁
  ████████▇▇▇▇▇▇▇▇▇▆▆▆▄▆▄▄▅▅▅▅▅▃▄▅▅▅▄▄▅▄▅▅▅▄▄▄▁▃▄▄▄▁▄▄▃▁▄▄▃▃ █
  1.1 μs       Histogram: log(frequency) by time     32.9 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [53]:
df = DataFrame("Variable"=>["Global", "Constant"], 
                "Median Time (μs)"=>[median(time_global.times) / 1000, median(time_constant.times) / 1000],
                "Speed comparison"=>[1.0, median(time_global.times)/median(time_constant.times)]);

print(df)

2×3 DataFrame
 Row │ Variable  Median Time (μs)  Speed comparison 
     │ String    Float64           Float64          
─────┼──────────────────────────────────────────────
   1 │ Global            111.0               1.0
   2 │ Constant            1.1478           96.7067

We can see from the table above that using a constant global variable (by specifying `const` before the variable name) speeds up the process by almost 100x than using a global variable. Now to see why this happens, we can use the `@code_warntype` macro to see the data types of our variables.

In [54]:
@code_warntype multiply_array(random_array)

Variables
  #self#::Core.Const(multiply_array)
  x::Vector{Float64}
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  y::Float64

Body::Any
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = x::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}::Tuple{Float64, Int64}
│         (y = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Any
│   %12 = (y * Main.p)::Any
│         (s = %11 + %12)
│         (@_3 = Base.iterate(%3, %10))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s


In [56]:
@code_warntype multiply_array2(random_array)

Variables
  #self#::Core.Const(multiply_array2)
  x::Vector{Float64}
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  y::Float64

Body::Float64
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = x::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}::Tuple{Float64, Int64}
│         (y = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Float64
│   %12 = (y * Main.p2)::Float64
│         (s = %11 + %12)
│         (@_3 = Base.iterate(%3, %10))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s


We can see from above that the function `multiply_array` which uses a global variable `p` is type-unstable making it run slow. When we declared the variable as a constant global variable (`p2` which eas used in `multiply_array2`), we see that the type instability was solved (the `Any` data type was solved).

### KR2 Replicated the naive implementation of the polynomial in the textbook.

In [8]:
function naive_polynomial(x, a...) #can i change a... into vector{a}
    p = zero(x)
    for i = 1:length(a)
        p = p + a[i] * x^(i-1)
    end
    return p
end

naive_polynomial (generic function with 1 method)

In [9]:
f_naive(x) = naive_polynomial(x, 1,2,3,4,5,6,7,8,9)

f_naive (generic function with 1 method)

In [10]:
x = 3.5
f_naive(x)

271125.95703125

In [18]:
time_naive = @benchmark f_naive($x)

BenchmarkTools.Trial: 10000 samples with 737 evaluations.
 Range (min … max):  171.342 ns … 335.125 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     258.493 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   511.756 ns ±   3.967 μs  ┊ GC (mean ± σ):  0.33% ± 1.74%

  ▆█▅▄▃▃▂▂▁▁▁▁▁▁                                                ▂
  █████████████████▇▇▇▇▇▆▇▆▆▆▆▆▇▆▆▄▆▅▆▆▅▅▅▅▃▆▅▅▄▅▄▄▅▅▁▃▄▄▄▁▅▄▄▄ █
  171 ns        Histogram: log(frequency) by time       3.67 μs <

 Memory estimate: 32 bytes, allocs estimate: 2.

This naive implementation of solving a polynomial has a slow runtime because we need to compute multiple exponentiation which is a particularly expensive operation.

### KR3: Replicated the naive implementation of the Horner’s method for the same polynomial.

To make things faster, we use the Horner's method instead, which uses multiplication of variables instead of exponentiation.

In [12]:
function poly_horner(x, a...)
    b = zero(x)
    for i = length(a):-1:1
        b = a[i] + b * x
    end
    return b
end

poly_horner (generic function with 1 method)

In [13]:
f_horner_naive(x) = poly_horner(x,1,2,3,4,5,6,7,8,9)

f_horner_naive (generic function with 1 method)

In [19]:
time_horner = @benchmark f_horner_naive($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):   4.887 ns …   8.941 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      8.522 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.938 ns ± 160.079 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆▆▃█▆              ▁                                         ▂
  ██████▇▆▅▄▄▆▆▆█▅▅▅▇█▆▄▅▅▆▆▅▄▅▄▄▄▅▄▄▁▄▃▅▄▄▁▃▄▃▄▄▁▄▃▄▄▁▄▃▄▄▄▁▄ █
  4.89 ns       Histogram: log(frequency) by time        64 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Let's compare this naive implementation of Horner's method to the solution in KR2.

In [59]:
compare = DataFrame("Method"=>["Naive", "Horner"], "Raw Time (ns)"=>[med_time_naive,med_time_horner],
"Speed comparison"=>[1.0, med_time_naive/med_time_horner]);

print(compare)

2×3 DataFrame
 Row │ Method  Raw Time (ns)  Speed comparison 
     │ String  Float64        Float64          
─────┼─────────────────────────────────────────
   1 │ Naive         258.493            1.0
   2 │ Horner          8.522           30.3324

We can see that that using Horner's method improves our algorithm by 30x in terms of evaluation speed.

### KR4: Replicated the macro implementation of the Horner’s method of the same polynomial

Since the coefficients of the polynomials are just constants, we can actually write a macro that optimizes the Horner's method. So as long as we don't change the coefficients of our polynomials, our defined macro below still works. Using this macro would then run an optimized expression making our evaluation speed faster.

In [15]:
macro horner(x,p...)
    ex = esc(p[end])
    for i = length(p):-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end


@horner (macro with 1 method)

In [16]:
f_horner_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

f_horner_macro (generic function with 1 method)

In [25]:
time_horner_macro = @benchmark f_horner_macro($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.045 ns … 212.760 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.065 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.107 ns ±   2.367 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ▂           █                                     
  ▂▃▂▂▄▅▅▅▆▅▆█▅▆▆▆▆▆▆▆▇▇██▇▆▅▅▄▃▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  0.045 ns        Histogram: frequency by time       0.109 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Now, let's compare this method to the naive implementation of the polynomial and the naive implementation of the Horner's method.

In [60]:
compare2 = DataFrame("Method"=>["Naive", "Horner Macro"], 
"Raw Time (ns)"=>[med_time_naive,med_time_horner_macro],
"Speed comparison"=>[1.0, med_time_naive/med_time_horner_macro]);

compare3 = DataFrame("Method"=>["Horner", "Horner Macro"], 
"Raw Time (ns)"=>[med_time_horner,med_time_horner_macro],
"Speed comparison"=>[1.0, med_time_horner/med_time_horner_macro]);

println(compare2)
println()
println(compare3)

2×3 DataFrame
 Row │ Method  Raw Time (ns)  Speed comparison 
     │ String  Float64        Float64          
─────┼─────────────────────────────────────────
   1 │ Naive         258.493              1.0
   2 │ Horner          0.065           3976.81

2×3 DataFrame
 Row │ Method        Raw Time (ns)  Speed comparison 
     │ String        Float64        Float64          
─────┼───────────────────────────────────────────────
   1 │ Horner                8.522             1.0
   2 │ Horner Macro          0.065           131.108


We can see that the Horner's (macro version) is faster than the naive implementation of the polynomial by approximately 4000x and it is 131 times faster when compared to the naive implementation of the Horner's method. We see here the importance of looking at our code and see to it that we should always make sure that we are not computing constants every time, and if we should use macro whenever possible.

### KR5: Table showing how many minutes will the function evaluations in both KR3 and KR4 be reduced if KR2 requires 24 hours of runtime.

In [27]:
med_time_naive = median(time_naive.times)
med_time_horner = median(time_horner.times)
med_time_horner_macro = median(time_horner_macro.times);

In [57]:
speedup1 = med_time_naive / med_time_horner
speedup2 = med_time_naive / med_time_horner_macro

table = DataFrame("Method"=>["Naive", "Horner", "Horner Macro"], "Raw Time (ns)"=>[med_time_naive,med_time_horner,med_time_horner_macro],
"Speed comparison"=>[1.0, speedup1, speedup2]);

In [41]:
print(table)

3×3 DataFrame
 Row │ Method        Raw Time (ns)  Speed comparison 
     │ String        Float64        Float64          
─────┼───────────────────────────────────────────────
   1 │ Naive               258.493            1.0
   2 │ Horner                8.522           30.3324
   3 │ Horner Macro          0.065         3976.81

To repeat and summarize the discussions from KR2 to KR4, we see from the summarized table above that the Horner's method performs 30x faster compared to the naive polynomial method, while the macro version of the Horner's method is almost 4000x faster that the naive one. Now assuming that our naive implementation of the polynomial takes 24 hours (1440 minutes) to run, let us check how many minutes it would take for the horner's method and its macro version to finish the same task.

In [43]:
table2 = DataFrame("Method"=>["Naive", "Horner", "Horner Macro"], 
                    "Time (hours)"=>[24, 24/speedup1, 24/speedup2], 
                    "Time (mins)"=>[(24*60), (24*60)/speedup1, (24*60)/speedup2]);

In [44]:
print(table2)

3×3 DataFrame
 Row │ Method        Time (hours)  Time (mins) 
     │ String        Float64       Float64     
─────┼─────────────────────────────────────────
   1 │ Naive          24.0         1440.0
   2 │ Horner          0.791234      47.474
   3 │ Horner Macro    0.00603499     0.362099

We see that using the naive implementation of the Horner's method would finish running after ~47.5 minutes or less than an hour while the macro implementation of the same method would be done in less than a minute (~22 seconds).